# Notebook 10: Frequency-Domain Specifications and Loop Shaping
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook explores frequency-domain specifications and loop shaping approaches for control system design, covering concepts like gain and phase margins, performance limitations, and systematic control synthesis methods.

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

## Learning Objectives

After completing this material, you should be able to:

1. Understand and specify control requirements in the frequency domain:
   - Express command tracking requirements 
   - Specify disturbance rejection requirements
   - Define noise rejection specifications

2. Use loop shaping techniques for control design:
   - Apply proportional control effectively
   - Design lead compensation for phase margin improvement
   - Design lag compensation for steady-state error reduction
   - Combine compensation elements systematically

3. Understand fundamental performance limitations:
   - Recognize the waterbed effect
   - Handle constraints from non-minimum phase zeros
   - Deal with limitations from unstable poles
   - Apply Bode's integral theorem

## Required Packages

Run the following cell to import required packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, FloatSlider
from IPython.display import display, clear_output, HTML
from scipy import signal
import control as ctrl

## 1. Frequency-Domain Specifications

### 1.1 Introduction 

Frequency-domain specifications provide a powerful way to express control requirements. The key idea is to separate different control objectives by frequency:

- Low frequencies: Command tracking and disturbance rejection
- High frequencies: Noise rejection 
- Mid frequencies: Stability margins and robustness

For a closed-loop system, we work with two key transfer functions:

1. Sensitivity function: $S(s) = \frac{1}{1 + L(s)}$
2. Complementary sensitivity function: $T(s) = \frac{L(s)}{1 + L(s)}$

where $L(s) = P(s)C(s)$ is the loop transfer function.

Key specifications are typically written as:

1. Command tracking/disturbance rejection:
   - $|S(j\omega)| \ll 1$ at low frequencies
   - Example: "ensure commands are tracked with max 10% error up to 10Hz"

2. Noise rejection:
   - $|T(j\omega)| \ll 1$ at high frequencies  
   - Example: "ensure noise is reduced by factor of 10 at frequencies above 100Hz"

Let's create an interactive visualization to understand these specifications:

In [ ]:
def create_sensitivity_plot(crossover_freq=1, low_freq_bound=0.1, high_freq_bound=10):
    # Create frequency points
    w = np.logspace(-2, 4, 1000)
    
    # Create simple loop transfer function L(s) = wc/s
    wc = crossover_freq
    L = wc/w*1j
    
    # Calculate sensitivity and complementary sensitivity
    S = 1/(1 + L)
    T = L/(1 + L)
    
    # Convert to dB
    S_db = 20*np.log10(np.abs(S))
    T_db = 20*np.log10(np.abs(T))
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.semilogx(w, S_db, 'b-', label='|S(jω)| (Sensitivity)')
    ax.semilogx(w, T_db, 'r--', label='|T(jω)| (Comp. Sensitivity)')
    
    # Add specification bounds
    w_low = w[w <= low_freq_bound]
    w_high = w[w >= high_freq_bound]
    ax.fill_between(w_low, -20*np.ones_like(w_low), 20*np.ones_like(w_low), 
                   color='red', alpha=0.1, label='Low freq bound')
    ax.fill_between(w_high, -20*np.ones_like(w_high), 20*np.ones_like(w_high),
                   color='blue', alpha=0.1, label='High freq bound')
    
    ax.grid(True)
    ax.set_xlabel('Frequency (rad/s)')
    ax.set_ylabel('Magnitude (dB)')
    ax.set_title('Sensitivity Functions')
    ax.legend()
    ax.set_ylim(-40, 20)
    plt.show()

# Create interactive controls
wc_slider = FloatSlider(value=1, min=0.1, max=10, description='Crossover freq:')
low_bound = FloatSlider(value=0.1, min=0.01, max=1, description='Low freq bound:')
high_bound = FloatSlider(value=10, min=1, max=100, description='High freq bound:')

interactive_output(create_sensitivity_plot, 
                  {'crossover_freq': wc_slider,
                   'low_freq_bound': low_bound,
                   'high_freq_bound': high_bound})
display(wc_slider, low_bound, high_bound)

### 1.3 Translating to Open-Loop Specifications

The closed-loop specifications can be translated to requirements on the open-loop transfer function $L(s)$:

1. Command tracking/disturbance rejection requirement:
   - $|S(j\omega)| \cdot |W_1(j\omega)| < 1$ 
   - This translates to $|L(j\omega)| > |W_1(j\omega)|$

2. Noise rejection requirement:
   - $|T(j\omega)| \cdot |W_2(j\omega)| < 1$
   - This translates to $|L(j\omega)| < |W_2(j\omega)|^{-1}$

These create an "obstacle course" on the Bode plot through which we must navigate $L(j\omega)$. Let's visualize this:

In [ ]:
def plot_obstacle_course(W1_gain=100, W2_gain=0.01):
    # Create frequency points
    w = np.logspace(-2, 4, 1000)
    
    # Create weighting functions
    W1 = W1_gain/(w + 0.1)  # Low frequency bound
    W2 = W2_gain*(w + 10)   # High frequency bound
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.semilogx(w, 20*np.log10(np.abs(W1)), 'r-', label='Low freq bound (W₁)')
    ax.semilogx(w, -20*np.log10(np.abs(W2)), 'b--', label='High freq bound (1/W₂)')
    
    # Add shaded "forbidden" regions
    ax.fill_between(w, 20*np.log10(np.abs(W1)), -100*np.ones_like(w),
                   color='red', alpha=0.1)
    ax.fill_between(w, 100*np.ones_like(w), -20*np.log10(np.abs(W2)),
                   color='blue', alpha=0.1)
    
    ax.grid(True)
    ax.set_xlabel('Frequency (rad/s)')
    ax.set_ylabel('Magnitude (dB)')
    ax.set_title('Bode Plot Obstacle Course')
    ax.legend()
    ax.set_ylim(-60, 60)
    plt.show()

# Create interactive controls
W1_slider = FloatSlider(value=100, min=1, max=1000, description='W₁ gain:')
W2_slider = FloatSlider(value=0.01, min=0.001, max=0.1, description='W₂ gain:')

interactive_output(plot_obstacle_course,
                  {'W1_gain': W1_slider,
                   'W2_gain': W2_slider})
display(W1_slider, W2_slider)